# Stop/Start feature test

In [ ]:
import sys
sys.path.append('../src')
import os
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(font_scale=2, style='whitegrid')

%matplotlib notebook

import FASTSim
import importlib
importlib.reload(FASTSim)

In [ ]:
cyc = FASTSim.get_standard_cycle("UDDS")

In [ ]:
veh0 = FASTSim.get_veh(6)
veh1 = FASTSim.get_veh(27)

### Run FASTSim

The "sim_drive" function takes the drive cycle and vehicle models defined above as inputs. The output is a dictionary of time series and scalar values described the simulation results. Typically of interest is the "gge" key, which is an array of time series energy consumption data at each time step in the drive cycle. Additionally, to add a result from the simulator to the output dictionary, a user can modify the sim_drive_sub function source code to include the desired field.

If running FASTSim in batch over many drive cycles, the output from "sim_drive" can be written to files or database for batch post-processing. 

In [ ]:
output0 = FASTSim.sim_drive(cyc, veh0)
output1 = FASTSim.sim_drive(cyc, veh1)

### Results

In [ ]:
df0 = pd.DataFrame.from_dict(output0)[['soc','fcKwInAch']]
df0['speed'] = cyc['cycMps'] * 2.23694  # Convert mps to mph

df1 = pd.DataFrame.from_dict(output1)[['soc','fcKwInAch']]
df1['speed'] = cyc['cycMps'] * 2.23694  # Convert mps to mph

In [ ]:
fig, ax = plt.subplots(figsize=(28, 10))
kw_line = df0.loc[0:400, :].fcKwInAch.plot(ax=ax, color='xkcd:bluish', label='CV Pw', linestyle='-')
kw_line = df1.loc[0:400, :].fcKwInAch.plot(ax=ax, color='xkcd:pale red', label='HEV Pw', linestyle='-')

# ax2 = ax.twinx()
# speed_line = df0.loc[0:400, :].speed.plot(color='xkcd:bluish', ax=ax2, label='CV Pw', linestyle='--')
# speed_line = df1.loc[0:400, :].speed.plot(color='xkcd:pale red', ax=ax2, label='Speed Pw', linestyle='--')

ax.set_xlabel('Cycle Time [s]', weight='bold')
ax.set_ylabel('Engine Input Power [kW]', weight='bold', color='xkcd:bluish')
ax.tick_params('y', colors='xkcd:bluish')
ax.legend()

# ax2.set_ylabel('Speed [MPH]', weight='bold', color='xkcd:pale red')
# ax2.grid(False)
# ax2.tick_params('y', colors='xkcd:pale red')

In [ ]:
fig, ax = plt.subplots(figsize=(28, 10))
kw_line = df1.loc[0:400, :].soc.plot(ax=ax, color='xkcd:bluish', label='SOC')

ax.set_xlabel('Cycle Time [s]', weight='bold')
ax.set_ylabel('SOC', weight='bold', color='xkcd:bluish')
ax.tick_params('y', colors='xkcd:bluish')